In [0]:
import os
import sys
import pickle
import glob
import numpy as np
import pandas as pd
import collections
import scipy.misc
import matplotlib.pyplot as plt
from skimage.transform import resize
import imageio
from collections import Counter

# possible neural network architectures
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet

# and corresponding pre-processing
from keras.applications.imagenet_utils import preprocess_input

# Keras layers and utils
import keras.backend as K
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Flatten, Dense, BatchNormalization, AveragePooling2D
from keras.layers import Conv2D, GlobalAveragePooling2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Reshape
from keras import optimizers
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# transformations
datagen =ImageDataGenerator(rotation_range=40,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.01,
                               zoom_range=[0.9, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                           )

def get_filenames_and_classes(csv_path):
    df = pd.read_csv(csv_path, sep='\t', index_col=0)
    path_classes, train_classes = np.array([]), np.array([])
    for path, classes in zip(df.index, df.iloc[:,0]):
        train_classes = np.append(train_classes,int(classes))
        path_classes = np.append(path_classes,str(path))
    return path_classes, train_classes

def open_jpg(img):
    return imageio.imread(img)

# Create balanced generators
def get_batches(path_classes,train_classes):
    while True:
        batch_input = np.zeros((32,224,224,3))
        batch_output = np.zeros((32,2))
        cond = 0
        for i in range(32):
            temp = 0
            index = np.random.permutation(len(train_classes))
            while temp == 0:
                if (int(train_classes[(index[i])]) == 1) & (cond%2==0):
                    temp = 1
                    cond = cond + 1
                    img = open_jpg(path_classes[(index[i])])
                    img = img.reshape((1,) + img.shape)
                    img = datagen.flow(img, batch_size=1).next().reshape(750,750,3)
                    imgr = resize(img, (224, 224), anti_aliasing=True)
                    batch_input[i] = imgr
                    batch_output[i][0] = 1
                elif (int(train_classes[(index[i])]) == 2) & (cond%2==1):
                    temp = 1
                    cond = cond + 1
                    img = open_jpg(path_classes[(index[i])])
                    img = img.reshape((1,) + img.shape)
                    img = datagen.flow(img, batch_size=1).next().reshape(750,750,3)
                    imgr = resize(img, (224, 224), anti_aliasing=True)
                    batch_input[i] = imgr
                    batch_output[i][1] = 1
                else:
                    index = np.random.permutation(len(train_classes))
        yield batch_input, batch_output

In [0]:
def main():
    #--------train batches
    path_classes, train_classes = np.array([]), np.array([])
    for pref in ['ZT199', 'ZT204', 'ZT111']:
        #change this for your path
        path_csv = "/home/fabianleon/Documentos/gleason_CNN-master/dataset/tma_info/" + str(pref) + "_pgleason_scores.csv"
        print(path_csv)
        path, classes = get_filenames_and_classes(path_csv)
        path_classes = np.append(path_classes,path)
        train_classes = np.append(train_classes,classes)
    train_batches = get_batches(path_classes,train_classes)
    print(collections.Counter(train_classes))
    #--------validation batches
    pathv_classes, validation_classes = np.array([]), np.array([])
    #change this for your path
    path_csv = "/home/fabianleon/Documentos/gleason_CNN-master/dataset/tma_info/ZT76_pgleason_scores.csv"
    print(path_csv)
    pathv_classes, validation_classes = get_filenames_and_classes(path_csv)
    validation_batches = get_batches(pathv_classes,validation_classes)
    print(collections.Counter(validation_classes)) 
    n_class = 2
    #change this for your path
    mpath = '/home/fabianleon/Documentos/gleason_CNN-master/model_weights/class1y2incep'
    # create model
    if not os.path.exists(mpath):
        os.makedirs(mpath)
    model_weights = os.path.join(mpath, 'model_{epoch:02d}.h5')
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg')
    x_top = base_model.output
    x_out = Dense(n_class, name='output', activation='softmax')(x_top)
    model = Model(base_model.input, x_out)
    model.summary()
    # train only softmax
    for layer in base_model.layers:
        layer.trainable = False                   
    model.compile(optimizer=optimizers.Adam(lr=0.001),
                  loss='categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit_generator(
                generator=train_batches,
		            use_multiprocessing=True,
                steps_per_epoch=200,
                epochs=5,
                validation_data=validation_batches,
                validation_steps=100,
                verbose=1
    )
    # train all layers
    for layer in model.layers:
        layer.trainable = True

    model.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
                  loss='categorical_crossentropy', metrics = ['accuracy'])
    checkpoint = ModelCheckpoint(filepath=model_weights, save_best_only=False, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=0.00001)

    history = model.fit_generator(
                generator=train_batches,
		            use_multiprocessing=True,
                steps_per_epoch=200,
                epochs=50,
                callbacks=[checkpoint, reduce_lr],
                validation_data=validation_batches,
                validation_steps=100,
                verbose=1
    )
    with open(os.path.join(mpath, 'history.pkl'), 'wb') as history_f:
        pickle.dump(history.history, history_f, protocol=2)
    
if __name__ == "__main__":
    main()
